In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
init_notebook_mode(connected = True)

In [3]:
# Function defined to check medata of a dataframe
def master_dataframe(dataframe):
    df_metadata = pd.DataFrame({'Datatype': dataframe.dtypes,
                                "Null Values": dataframe.isna().sum(),  
                                "Null %": round(dataframe.isna().sum()/len(dataframe)*100, 2),
                                "No: Of Unique Values": dataframe.nunique()})
    
    df_describe = dataframe.describe(include='all').T
    
    df_metadata = df_metadata.join(df_describe)  

    return df_metadata

In [4]:
raw_data = pd.read_csv('../data/Admission_Predict.csv')

In [5]:
# Calling the function to check metadata.
master_dataframe(raw_data)

,Datatype,Null Values,Null %,No: Of Unique Values,count,mean,std,min,25%,50%,75%,max
Serial No.,int64,0,0.0,400,400.0,200.500000,115.614301,1.00,100.75,200.50,300.2500,400.00
GRE Score,int64,0,0.0,49,400.0,316.807500,11.473646,290.00,308.00,317.00,325.0000,340.00
TOEFL Score,int64,0,0.0,29,400.0,107.410000,6.069514,92.00,103.00,107.00,112.0000,120.00
University Rating,int64,0,0.0,5,400.0,3.087500,1.143728,1.00,2.00,3.00,4.0000,5.00
SOP,float64,0,0.0,9,400.0,3.400000,1.006869,1.00,2.50,3.50,4.0000,5.00
LOR,float64,0,0.0,9,400.0,3.452500,0.898478,1.00,3.00,3.50,4.0000,5.00
CGPA,float64,0,0.0,168,400.0,8.598925,0.596317,6.80,8.17,8.61,9.0625,9.92
Research,int64,0,0.0,2,400.0,0.547500,0.498362,0.00,0.00,1.00,1.0000,1.00
Chance of Admit,float64,0,0.0,60,400.0,0.724350,0.142609,0.34,0.64,0.73,0.8300,0.97


In [6]:
raw_data.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [7]:
# Droppig Serial No. from the data frame and removing extra space from column Chance of Admit
data_col_chng = raw_data.copy()
data_col_chng.drop(columns = ['Serial No.'], axis = 1, inplace = True)
data_col_chng.columns = data_col_chng.columns.str.strip()

In [8]:
data_col_chng.columns

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA',
       'Research', 'Chance of Admit'],
      dtype='object')

In [9]:
data_col_chng

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1,0.82
396,325,107,3,3.0,3.5,9.11,1,0.84
397,330,116,4,5.0,4.5,9.45,1,0.91
398,312,103,3,3.5,4.0,8.78,0,0.67


In [10]:
grp_UR = data_col_chng.groupby(by=['University Rating'])["Chance of Admit"].mean().reset_index()
grp_UR['Chance of Admit'] = round(grp_UR['Chance of Admit']*100, 2)
grp_UR.sort_values(by=['Chance of Admit'], inplace = True)

data = [go.Bar(x = grp_UR['University Rating'], 
               y = grp_UR['Chance of Admit'], 
               text = grp_UR['Chance of Admit'],
               textposition = 'inside',
               marker = dict(line = {'color': 'black', 'width': 0.5}))]

layout = go.Layout(title = dict(text = 'Percentage Chance Of Admission By University Rating', 
                                font = {'size': 20},
                                x = 0.5, 
                                y = 0.88, 
                                xanchor = 'center', 
                                yanchor = 'top'), 
                   xaxis = dict(title = 'University Rating', 
                                titlefont = {'size': 16}), 
                   yaxis = dict(title = 'Percentage Chance Of Admission', 
                                titlefont = {'size': 16}, 
                                range = [0, 100]))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)


In [11]:
grp_LOR= data_col_chng.groupby(by=['LOR'])["Chance of Admit"].mean().reset_index()
grp_LOR['Chance of Admit'] = round(grp_LOR['Chance of Admit']*100, 2)
grp_LOR.sort_values(by=['Chance of Admit'], inplace = True)

data = [go.Bar(x = grp_LOR['LOR'], 
               y = grp_LOR['Chance of Admit'], 
               text = grp_LOR['Chance of Admit'],
               textposition = 'inside',
               marker = dict(line = {'color': 'black', 'width': 0.5}))]

layout = go.Layout(title = dict(text = 'Percentage Chance Of Admission By Letter Of Recommendation', 
                                font = {'size': 20},
                                x = 0.5, 
                                y = 0.88, 
                                xanchor = 'center', 
                                yanchor = 'top'), 
                   xaxis = dict(title = 'LOR', 
                                titlefont = {'size': 16}), 
                   yaxis = dict(title = 'Percentage Chance Of Admission', 
                                titlefont = {'size': 16}, 
                                range = [0, 100]))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)


In [12]:
grp_SOP = data_col_chng.groupby(by=['SOP'])["Chance of Admit"].mean().reset_index()
grp_SOP['Chance of Admit'] = round(grp_SOP['Chance of Admit']*100, 2)
grp_SOP.sort_values(by=['Chance of Admit'], inplace = True)

data = [go.Bar(x = grp_SOP['SOP'], 
               y = grp_SOP['Chance of Admit'], 
               text = grp_SOP['Chance of Admit'],
               textposition = 'inside',
               marker = dict(line = {'color': 'black', 'width': 0.5}))]

layout = go.Layout(title = dict(text = 'Percentage Chance Of Admission By SOP', 
                                font = {'size': 20},
                                x = 0.5, 
                                y = 0.88, 
                                xanchor = 'center', 
                                yanchor = 'top'), 
                   xaxis = dict(title = 'SOP', 
                                titlefont = {'size': 16}), 
                   yaxis = dict(title = 'Percentage Chance Of Admission', 
                                titlefont = {'size': 16}, 
                                range = [0, 100]))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)


In [13]:
grp_Res = data_col_chng.groupby(by=['Research'])["Chance of Admit"].mean().reset_index()
grp_Res['Chance of Admit'] = round(grp_Res['Chance of Admit']*100, 2)
grp_Res.sort_values(by=['Chance of Admit'], inplace = True)

data = [go.Pie(values = grp_Res['Chance of Admit'], 
               )]

layout = go.Layout(title = dict(text = 'Percentage Chance Of Admission By Research', 
                                font = {'size': 20},
                                x = 0.5, 
                                y = 0.88, 
                                xanchor = 'center', 
                                yanchor = 'top')
                  )

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [14]:
data = [go.Scatter(x = data_col_chng['GRE Score'], 
                   y = data_col_chng['Chance of Admit']*100, 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = data_col_chng['SOP'], 
                                 showscale = True))]

layout = go.Layout(title = dict(text = 'GRE Score Vs Chance Of Admit - SOP', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'GRE Score', 
                                titlefont = {'size': 16}),
                   yaxis = dict(title = 'Chance Of Admit', 
                                titlefont = {'size': 16}),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [15]:
data = [go.Scatter(x = data_col_chng['CGPA'], 
                   y = data_col_chng['Chance of Admit']*100, 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = data_col_chng['University Rating'], 
                                 showscale = True))]

layout = go.Layout(title = dict(text = 'CGPA Vs Chance Of Admit - University Rating', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'CGPA', 
                                titlefont = {'size': 16}),
                   yaxis = dict(title = 'Chance Of Admit', 
                                titlefont = {'size': 16}),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [16]:
data = [go.Scatter(x = data_col_chng['TOEFL Score'], 
                   y = data_col_chng['Chance of Admit']*100, 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = data_col_chng['LOR'], 
                                 showscale = True))]

layout = go.Layout(title = dict(text = 'TOEFL Vs Chance Of Admit - LOR', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'TOEFL', 
                                titlefont = {'size': 16}),
                   yaxis = dict(title = 'Chance Of Admit', 
                                titlefont = {'size': 16}),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [17]:
# Checking for outliers and the scale of values of variables.
data = []
for i in data_col_chng.columns[:-1]:
    data .append(go.Box(y = data_col_chng[i], 
                        marker = dict(line = {'color': 'black', 'width': 0.5}, 
                                      opacity = 0.5), 
                        name = str(i), 
                        boxpoints = 'outliers', 
                        notched = True))

layout = go.Layout(title = dict(text = 'Checking the range and scale of all the features.', 
                                x = 0.5, 
                                y = 0.88))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [18]:
from sklearn.preprocessing import StandardScaler
data_scaled_using_ss = data_col_chng.copy()
ss_for_df = StandardScaler()
data_scaled_using_ss[['GRE Score', 'TOEFL Score']] = ss_for_df.fit_transform(data_col_chng[['GRE Score', 'TOEFL Score']])

In [19]:
data = []
for i in data_scaled_using_ss.columns[:-1]:
    data .append(go.Box(y = data_scaled_using_ss[i], 
                        marker = dict(line = {'color': 'black', 'width': 0.5}, 
                                      opacity = 0.5), 
                        name = str(i), 
                        boxpoints = 'outliers', 
                        notched = True))

layout = go.Layout(title = dict(text = 'Checking the range and scale of all the features.', 
                                x = 0.5, 
                                y = 0.88))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [20]:
data_col_chng.columns[:-1]

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA',
       'Research'],
      dtype='object')

In [21]:
from sklearn.preprocessing import MinMaxScaler
data_scaled_using_mms = data_col_chng.copy()
mms_for_df = MinMaxScaler()
data_scaled_using_mms[data_scaled_using_mms.columns[:-1]] = mms_for_df.fit_transform(data_col_chng[data_scaled_using_mms.columns[:-1]]) 

In [22]:
data = []
for i in data_scaled_using_mms.columns[:-1]:
    data .append(go.Box(y = data_scaled_using_mms[i], 
                        marker = dict(line = {'color': 'black', 'width': 0.5}, 
                                      opacity = 0.5), 
                        name = str(i), 
                        boxpoints = 'outliers', 
                        notched = True))

layout = go.Layout(title = dict(text = 'Checking the range and scale of all the features.', 
                                x = 0.5, 
                                y = 0.88))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [23]:
# Lets split out dependent and independent variables.
X = data_col_chng.iloc[:, :-1].values
y = data_col_chng.iloc[:, -1].values
print('Shape of X: ', X.shape)
print('Shape of y: ', y.shape)

Shape of X:  (400, 7)
Shape of y:  (400,)


In [24]:
# Lets split our dataset into train & test data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print('Shape of X_train: ', X_train.shape)
print('Shape of X_test: ', X_test.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of y_test: ', y_test.shape)

Shape of X_train:  (320, 7)
Shape of X_test:  (80, 7)
Shape of y_train:  (320,)
Shape of y_test:  (80,)


In [25]:
X_train_ss_scaled = X_train
X_test_ss_scaled = X_test
X_train_mms_scaled = X_train
X_test_mms_scaled = X_test

y_train_ss_scaled = y_train
y_test_ss_scaled = y_test
y_train_mms_scaled = y_train
y_test_mms_scaled = y_test

In [26]:
# Lets scale our data using MinMaxScaler.
mms_train_test = MinMaxScaler()
X_train_mms_scaled[:, :-1] = mms_train_test.fit_transform(X_train[:, :-1])
X_test_mms_scaled[:, :-1] = mms_train_test.transform(X_test[:, :-1])

In [27]:
ss_train_test = StandardScaler()
X_train_ss_scaled[:, :-1] = ss_train_test.fit_transform(X_train[:, :-1])
X_test_ss_scaled[:, :-1] = ss_train_test.transform(X_test[:, :-1])

In [28]:
# Creating a Multi-Linear Regression Model with MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
l_regressor = LinearRegression()
l_regressor.fit(X_train_mms_scaled, y_train)
y_pred_l_reg = l_regressor.predict(X_test_mms_scaled)
print(r2_score(y_test, y_pred_l_reg))

0.8212082591486991


In [29]:
# Creating a Multi-Linear Regression Model with StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
l_regressor_ss = LinearRegression()
l_regressor_ss.fit(X_train_ss_scaled, y_train)
y_pred_l_reg_ss = l_regressor_ss.predict(X_test_ss_scaled)
print(r2_score(y_test, y_pred_l_reg_ss))

0.8212082591486991


In [30]:
data = [go.Scatter(x = y_test, 
                   y = y_pred_l_reg, 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = 'skyblue'))]

layout = go.Layout(title = dict(text = 'Linear Regression - Actual Values vs Predicted Values', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'Actual Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   yaxis = dict(title = 'Predicted Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [31]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg_mms = PolynomialFeatures(degree = 2)
X_poly_mms = poly_reg_mms.fit_transform(X_train_mms_scaled)
X_test_poly_mms = poly_reg_mms.transform(X_test_mms_scaled)
poly_regressor_mms = LinearRegression()
poly_regressor_mms.fit(X_poly_mms, y_train)
y_pred_poly_mms = poly_regressor_mms.predict(X_test_poly_mms)
print(r2_score(y_test, y_pred_poly_mms))

0.8253834707740503


In [32]:
poly_reg_ss = PolynomialFeatures(degree = 2)
X_poly_ss = poly_reg_ss.fit_transform(X_train_ss_scaled)
X_test_poly_ss = poly_reg_ss.transform(X_test_ss_scaled)
poly_regressor_ss = LinearRegression()
poly_regressor_ss.fit(X_poly_ss, y_train)
y_pred_poly_ss = poly_regressor_ss.predict(X_test_poly_ss)
print(r2_score(y_test, y_pred_poly_ss))

0.8253834707740503


In [33]:
data = [go.Scatter(x = y_test, 
                   y = y_pred_poly_ss, 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = 'skyblue'))]

layout = go.Layout(title = dict(text = 'Polynomial Regression - Actual Values vs Predicted Values', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'Actual Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   yaxis = dict(title = 'Predicted Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [34]:
mms_y_train = MinMaxScaler()
y_train_mms_scaled = mms_y_train.fit_transform(y_train.reshape(-1,1))

from sklearn.svm import SVR
svr_regressor_mms = SVR(kernel = 'rbf')
svr_regressor_mms.fit(X_train_mms_scaled, y_train_mms_scaled.ravel())
y_pred_svr_mms = svr_regressor_mms.predict(X_test_mms_scaled)
y_pred_svr_mms = mms_y_train.inverse_transform(y_pred_svr_mms.reshape(-1, 1))
print(r2_score(y_test, y_pred_svr_mms))

0.7797918733594855


In [35]:
ss_y_train = StandardScaler()
y_train_ss_scaled = ss_y_train.fit_transform(y_train.reshape(-1,1))

from sklearn.svm import SVR
svr_regressor_ss = SVR(kernel = 'rbf')
svr_regressor_ss.fit(X_train_ss_scaled, y_train_ss_scaled.ravel())
y_pred_svr_ss = svr_regressor_ss.predict(X_test_ss_scaled)
y_pred_svr_ss = ss_y_train.inverse_transform(y_pred_svr_ss.reshape(-1, 1))
print(r2_score(y_test, y_pred_svr_ss))

0.8094206571056967


In [36]:
data = [go.Scatter(x = y_test, 
                   y = y_pred_svr_ss.ravel(), 
                   mode = 'markers', 
                   marker = dict(symbol = 'circle', 
                                 size = 16, 
                                 color = 'skyblue'))]

layout = go.Layout(title = dict(text = 'SVR - Actual Values vs Predicted Values', 
                                x = 0.5, 
                                y = 0.88, 
                                font = {'size': 20}),
                   xaxis = dict(title = 'Actual Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   yaxis = dict(title = 'Predicted Value', 
                                titlefont = {'size': 16}, 
                                range = [0, 1]),
                   template = 'plotly_dark')

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [37]:
# Decision Tree Regression with MinMaxScaled Data
from sklearn.tree import DecisionTreeRegressor
dtr_scaled_mms = DecisionTreeRegressor(random_state = 0)
dtr_scaled_mms.fit(X_train_mms_scaled, y_train)
y_pred_dtrs_mms = dtr_scaled_mms.predict(X_test_mms_scaled)
print(r2_score(y_test, y_pred_dtrs_mms))

0.6392316791664523


In [38]:
# Decision Tree Regression with StandardScaled Data
dtr_scaled_ss = DecisionTreeRegressor(random_state = 0)
dtr_scaled_ss.fit(X_train_ss_scaled, y_train)
y_pred_dtrs_ss = dtr_scaled_ss.predict(X_test_ss_scaled)
print(r2_score(y_test, y_pred_dtrs_ss))

0.6392316791664523


In [39]:
# Decision Tree Regression with unscaled data
from sklearn.tree import DecisionTreeRegressor
dtr_unscaled = DecisionTreeRegressor(random_state = 0)
dtr_unscaled.fit(X_train, y_train)
y_pred_dtr = dtr_unscaled.predict(X_test)
print(r2_score(y_test, y_pred_dtr))

0.6392316791664523


In [40]:
# Random Forest model with data scaled with MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
rf_mms_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_mms_regressor.fit(X_train_mms_scaled, y_train)
y_pred_rf_mms = rf_mms_regressor.predict(X_test_mms_scaled)
print(r2_score(y_test, y_pred_rf_mms))

0.7994010994164078


In [41]:
# Random Forest model with data scaled with StandardScaled Data
rf_ss_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_ss_regressor.fit(X_train_ss_scaled, y_train)
y_pred_rf_ss = rf_ss_regressor.predict(X_test_ss_scaled)
print(r2_score(y_test, y_pred_rf_ss))

0.7994010994164078


In [42]:
# Random Forest model with unscaled data
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X_train, y_train)
y_pred_rf = rf_regressor.predict(X_test)
print(r2_score(y_test, y_pred_rf))

0.7994010994164078
